In [3]:
import torch 
from torchvision import datasets, transforms
import torch.nn as nn 
import torch.optim as optim 

transform = transforms.Compose([
    transforms.ToTensor(), # convert to tensor,
    transforms.Normalize((0.2860,), (0.3530,)), # mean, standard deviation  (x - mean) / std for each image x 
]
)

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)



In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes = 10): 
        super().__init__()
        # We split our NN in two stages: 
        #   1. generate features / convolutional transformation
        #   2. classify them using FFNN 
        self.features = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, padding=1), # 28x28 -> 28x28 output
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2), # Taking maximum of 2x2 slider 28x28 -> 14x14 

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 14x14 -> 14x14
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), # 14x14 -> 7x7

            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 7x7 -> 7x7
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(), # 2D -> 1D
            nn.Linear(128*7*7, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes), # output
        )
    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [6]:
model = CNN()
lr = 1e-3 # learning rate
loss = nn.CrossEntropyLoss() # because multi-class problem
optimizer = optim.Adam(model.parameters(), lr=lr)

n_epochs = 3

for epoch in range(n_epochs):
    model.train() # training mode 
    running_loss = 0.0 # loss per epoch 
    for images, labels in train_loader:
        optimizer.zero_grad() # reset the gradient 
        # forward step 
        outputs = model(images) # propagate the images
        curr_loss = loss(outputs, labels) 
        running_loss += curr_loss
        # backward
        curr_loss.backward() # gradients
        optimizer.step() # update weights and biases
    
    print(f"Epoch[{epoch +1} / {n_epochs}], Loss: {running_loss}")

Epoch[1 / 3], Loss: 352.3607177734375
Epoch[2 / 3], Loss: 241.72332763671875
Epoch[3 / 3], Loss: 204.60267639160156


In [7]:
# Evaluation 


model.eval() # setting the model to evaluation mode (implementation optimization)
correct = 0
total = 0

with torch.no_grad(): # we are not interested in gradient anymore 
    for images, labels in test_loader:
        outputs = model(images) # logits
        predicted = torch.max(outputs.data, 1)[-1] # argmax-> at which class we have maximum logit 
        total += labels.size(0) 
        correct += (predicted==labels).sum().item()

accuracy = correct/total
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 91.71%
